Data review and validation

In [20]:
import xarray as xr
import numpy as np  
import matplotlib.pyplot as plt 
import pandas as pd

In [23]:

# %%
file = "germany_2019.grib"

# Load 10m wind (analysis)

ds_u10 = xr.open_dataset(
    file,
    engine="cfgrib",
    backend_kwargs={"filter_by_keys": {
        "shortName": "10u",
        "typeOfLevel": "surface",
        "dataType": "an"
    }}
)

ds_v10 = xr.open_dataset(
    file,
    engine="cfgrib",
    backend_kwargs={"filter_by_keys": {
        "shortName": "10v",
        "typeOfLevel": "surface",
        "dataType": "an"
    }}
)

ds_wind = xr.merge([ds_u10, ds_v10])

# Load solar radiation (forecast accumulated field)
ds_ssrd = xr.open_dataset(
    file,
    engine="cfgrib",
    backend_kwargs={"filter_by_keys": {
        "shortName": "ssrd",
        "typeOfLevel": "surface",
        "dataType": "fc"
    }}
)


# Merge datasets
ds = xr.merge([ds_wind, ds_ssrd], compat="override")

print(ds)

/Users/sid/Library/Python/3.12/lib/python/site-packages/cfgrib/xarray_plugin.py:123: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/Users/sid/Library/Python/3.12/lib/python/site-packages/cfgrib/xarray_plugin.py:123: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the pres

<xarray.Dataset> Size: 664MB
Dimensions:     (time: 8761, latitude: 33, longitude: 41, step: 12)
Coordinates:
  * time        (time) datetime64[ns] 70kB 2018-12-31T18:00:00 ... 2019-12-31...
  * latitude    (latitude) float64 264B 55.0 54.75 54.5 ... 47.5 47.25 47.0
  * longitude   (longitude) float64 328B 5.0 5.25 5.5 5.75 ... 14.5 14.75 15.0
  * step        (step) timedelta64[ns] 96B 01:00:00 02:00:00 ... 12:00:00
    number      int64 8B 0
    surface     float64 8B 0.0
    valid_time  (time) datetime64[ns] 70kB NaT ... 2019-12-31T23:00:00
Data variables:
    u10         (time, latitude, longitude) float32 47MB nan nan ... 1.63 1.649
    v10         (time, latitude, longitude) float32 47MB nan nan ... -0.8819
    ssrd        (time, step, latitude, longitude) float32 569MB nan nan ... nan
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conven

In [24]:

ds= ds.sortby('time')   

# 4. Fill remaining NaN values (e.g., solar night)
ds = ds.fillna(0)

print (ds)


<xarray.Dataset> Size: 664MB
Dimensions:     (time: 8761, latitude: 33, longitude: 41, step: 12)
Coordinates:
  * time        (time) datetime64[ns] 70kB 2018-12-31T18:00:00 ... 2019-12-31...
  * latitude    (latitude) float64 264B 55.0 54.75 54.5 ... 47.5 47.25 47.0
  * longitude   (longitude) float64 328B 5.0 5.25 5.5 5.75 ... 14.5 14.75 15.0
  * step        (step) timedelta64[ns] 96B 01:00:00 02:00:00 ... 12:00:00
    number      int64 8B 0
    surface     float64 8B 0.0
    valid_time  (time) datetime64[ns] 70kB NaT ... 2019-12-31T23:00:00
Data variables:
    u10         (time, latitude, longitude) float32 47MB 0.0 0.0 ... 1.63 1.649
    v10         (time, latitude, longitude) float32 47MB 0.0 0.0 ... -0.8819
    ssrd        (time, step, latitude, longitude) float32 569MB 0.0 0.0 ... 0.0
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conven

In [25]:
# %%
ssrd_acc = ds['ssrd']
ssrd_diff = ssrd_acc.diff(dim='step')
ssrd_wm2 = ssrd_diff.isel(step=0) / 3600.0

# Use valid_time as the single time axis
#ssrd_wm2 = ssrd_wm2.assign_coords(time=ds["valid_time"].isel(step=0))
ssrd_wm2 = ssrd_wm2.swap_dims({"time": "time"})

ds = ds.assign(ssrd_wm2 = ssrd_wm2)

# %%
#ssrd_acc = ds['ssrd']
#ssrd_hourly_energy = ssrd_acc.diff(dim='step')
#ssrd_1h = ssrd_hourly_energy.isel(step=0)

# 4) Convert J/m² → W/m² (divide by seconds per hour)
#ssrd_wm2 = ssrd_1h / 3600.0

# 6) Add this corrected variable into dataset
ds = ds.drop_vars(['ssrd'])          # remove old accumulated radiation
ds = ds.drop_vars(['step'])          # remove step dimension now that it's not needed
ds = ds.assign(ssrd_wm2=ssrd_wm2)    # add correct radiation

# Calculate wind speed
ds['wind_speed'] = np.sqrt(ds['u10']**2 + ds['v10']**2) 
print(ds)


<xarray.Dataset> Size: 190MB
Dimensions:     (time: 8761, latitude: 33, longitude: 41)
Coordinates:
  * time        (time) datetime64[ns] 70kB 2018-12-31T18:00:00 ... 2019-12-31...
  * latitude    (latitude) float64 264B 55.0 54.75 54.5 ... 47.5 47.25 47.0
  * longitude   (longitude) float64 328B 5.0 5.25 5.5 5.75 ... 14.5 14.75 15.0
    number      int64 8B 0
    surface     float64 8B 0.0
    valid_time  (time) datetime64[ns] 70kB NaT ... 2019-12-31T23:00:00
    step        timedelta64[ns] 8B 02:00:00
Data variables:
    u10         (time, latitude, longitude) float32 47MB 0.0 0.0 ... 1.63 1.649
    v10         (time, latitude, longitude) float32 47MB 0.0 0.0 ... -0.8819
    ssrd_wm2    (time, latitude, longitude) float32 47MB 0.0 0.0 0.0 ... 0.0 0.0
    wind_speed  (time, latitude, longitude) float32 47MB 0.0 0.0 ... 2.19 1.87
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
 

In [28]:
# Select whole year timeline 
ds_4m = ds.sel(time=slice("2019-01-01", "2019-12-31"))

# 2) Compute spatial mean (average over all grid points)
ds_mean = ds_4m.mean(dim=["latitude", "longitude"], keep_attrs=True)

# 3) Convert to a Pandas DataFrame
df = ds_mean.to_dataframe()

# 4) Save to CSV
output_file = "germany_2019.csv"
df.to_csv(output_file)

print("Export complete:", output_file)
print(df.head())


Export complete: germany_2019.csv
                          u10       v10  number  surface          valid_time  \
time                                                                           
2019-01-01 00:00:00  4.397684  1.059329       0      0.0 2019-01-01 00:00:00   
2019-01-01 01:00:00  4.562312  1.122408       0      0.0 2019-01-01 01:00:00   
2019-01-01 02:00:00  4.839377  1.151029       0      0.0 2019-01-01 02:00:00   
2019-01-01 03:00:00  5.088492  1.059656       0      0.0 2019-01-01 03:00:00   
2019-01-01 04:00:00  5.329007  0.831390       0      0.0 2019-01-01 04:00:00   

                     ssrd_wm2            step  wind_speed  
time                                                       
2019-01-01 00:00:00       0.0 0 days 02:00:00    4.707259  
2019-01-01 01:00:00       0.0 0 days 02:00:00    4.885884  
2019-01-01 02:00:00       0.0 0 days 02:00:00    5.185383  
2019-01-01 03:00:00       0.0 0 days 02:00:00    5.462704  
2019-01-01 04:00:00       0.0 0 days 02:00:00